## Building SQL database for Illinois traffic data

In [1]:
import pandas as pd
from glob import glob
import sets
import datetime

/Users/along528/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: the sets module is deprecated
  app.launch_new_instance()


In [2]:
df_map = {}

# 2004 

In [3]:
tmpdfs = []
for filename in glob('/Users/along528/Dropbox/insight/project/datasets/traffic_il/2004*Data/*Statewide*/*.txt'):
    tmpdfs.append(pd.read_csv(filename,delimiter='~',parse_dates='DateAndTimeOfStop'))
df_map['2004'] = pd.concat(tmpdfs)
df_map['2004'] = df_map['2004'].reset_index(drop=True)
df_map['2004']

/Users/along528/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (12,20,21,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AgencyName  AgencyCode      DateAndTimeOfStop  \
0        Abingdon Police Department       13462   1/15/2004 7:20:00 AM   
1        Abingdon Police Department       13462   1/17/2004 9:20:00 PM   
2        Abingdon Police Department       13462  1/19/2004 12:58:00 AM   
3        Abingdon Police Department       13462   1/19/2004 2:13:00 AM   
4        Abingdon Police Department       13462   1/21/2004 9:52:00 PM   
5        Abingdon Police Department       13462   1/10/2004 7:40:00 PM   
6        Abingdon Police Department       13462   1/11/2004 1:50:00 AM   
7        Abingdon Police Department       13462    1/3/2004 8:30:00 PM   
8        Abingdon Police Department       13462   1/3/2004 11:43:00 PM   
9        Abingdon Police Department       13462   1/2/2004 10:40:00 PM   
10       Abingdon Police Department       13462    1/4/2004 8:52:00 PM   
11       Abingdon Police Department       13462  2/19/2004 11:57:00 PM   
12       Abingdon Police Department       13462   2/19/2004 8:30:00 AM   
13       Abingdon Police Department       13462    2/2/2004 1:23:00 PM   
14       Abingdon Police Department       13462    2/1/2004 2:06:00 AM   
15       Abingdon Police Department       13462    2/1/2004 2:45:00 AM   
16       Abingdon Police Department       13462   1/28/2004 8:25:00 PM   
17       Abingdon Police Department       13462  1/24/2004 12:37:00 PM   
18       Abingdon Police Department       13462  1/24/2004 11:25:00 AM   
19       Abingdon Police Department       13462  1/24/2004 11:18:00 AM   
20       Abingdon Police Department       13462    2/8/2004 6:21:00 PM   
21       Abingdon Police Department       13462    2/8/2004 9:46:00 PM   
22       Abingdon Police Department       13462    2/7/2004 3:25:00 PM   
23       Abingdon Police Department       13462   2/16/2004 9:58:00 AM   
24       Abingdon Police Department       13462   2/15/2004 9:20:00 AM   
25       Abingdon Police Department       13462   2/15/2004 8:31:00 AM   
26       Abingdon Police Department       13462   7/14/2004 2:10:00 PM   
27       Abingdon Police Department       13462  2/13/2004 10:54:00 PM   
28       Abingdon Police Department       13462   2/13/2004 6:26:00 PM   
29       Abingdon Police Department       13462    2/7/2004 1:05:00 AM   
...                             ...         ...                    ...   
2494633                CRETE POLICE       13951  7/30/2004 11:00:00 PM   
2494634                CRETE POLICE       13951   7/30/2004 8:55:00 PM   
2494635                CRETE POLICE       13951   7/31/2004 5:07:00 PM   
2494636                CRETE POLICE       13951   7/1/2004 11:49:00 PM   
2494637                CRETE POLICE       13951   6/2/2004 10:30:00 PM   
2494638                CRETE POLICE       13951    7/7/2004 8:00:00 PM   
2494639                CRETE POLICE       13951    7/7/2004 8:20:00 PM   
2494640                CRETE POLICE       13951   7/8/2004 11:00:00 PM   
2494641                CRETE POLICE       13951   7/8/2004 10:30:00 PM   
2494642                CRETE POLICE       13951   7/8/2004 10:00:00 PM   
2494643                CRETE POLICE       13951    7/8/2004 7:30:00 PM   
2494644                CRETE POLICE       13951    7/9/2004 7:53:00 PM   
2494645                CRETE POLICE       13951   7/11/2004 6:30:00 PM   
2494646                CRETE POLICE       13951  7/11/2004 11:15:00 PM   
2494647                CRETE POLICE       13951  7/18/2004 11:00:00 PM   
2494648                CRETE POLICE       13951   7/18/2004 8:00:00 PM   
2494649                CRETE POLICE       13951  7/18/2004 10:20:00 PM   
2494650                CRETE POLICE       13951   7/18/2004 5:50:00 PM   
2494651                CRETE POLICE       13951   7/26/2004 5:00:00 PM   
2494652                CRETE POLICE       13951   7/26/2004 7:55:00 PM   
2494653                CRETE POLICE       13951  7/26/2004 10:55:00 PM   
2494654                CRETE POLICE       13951   7/29/2004 1:15:00 AM   
2494655                CRETE 

In [8]:
def race_map(x):
    #if type(x)!='object': return x
    x = x.replace('Caucasian','white')
    x = x.replace('African American','black')
    x = x.replace('Hispanic','hispanic')
    x = x.replace('Asian/Pacific Islander','asian')
    x = x.replace('Native American/Alaskan','native_american')
    return x
def process_2004(df):
    #change race reporting format
    tmpdf = pd.DataFrame(df)
    tmprace = tmpdf['Race']
    tmprace = tmprace.fillna('other')
    tmprace = tmprace.map(race_map)
    tmpdf['Race'] = tmprace
    #parse datese
    #tmpdf['DateAndTimeOfStop'] = pd.to_datetime(tmpdf['DateAndTimeOfStop'])
    #tmpdf['year'] = tmpdf['DateAndTimeOfStop'].map(lambda x: x.year)
    #tmpdf['month'] = tmpdf['DateAndTimeOfStop'].map(lambda x: x.month)
    #fillna
    tmpdf = tmpdf.fillna(-1)
    #pluck out columns of interest
    tmpdf =  tmpdf[['AgencyName',
                       'AgencyCode',
                       'DriverYearOfBirth',
                       'DriverSex',
                       'Race',
                       'SearchConducted',
                       'ContrabandFound']]#, 'year', #'month']]
    #format agency title
    tmpdf['AgencyName'] = tmpdf['AgencyName'].map(str.title)
    #format column names
    tmpdf.rename(columns=str.lower,inplace=True)
    return tmpdf
df_map['2004'] = process_2004(df_map['2004'])

In [9]:
df_map['2004']

agencyname  agencycode  driveryearofbirth driversex  \
0        Abingdon Police Department       13462               1955      Male   
1        Abingdon Police Department       13462               1972      Male   
2        Abingdon Police Department       13462               1985      Male   
3        Abingdon Police Department       13462               1981      Male   
4        Abingdon Police Department       13462               1984      Male   
5        Abingdon Police Department       13462               1976      Male   
6        Abingdon Police Department       13462               1953      Male   
7        Abingdon Police Department       13462               1982      Male   
8        Abingdon Police Department       13462               1982      Male   
9        Abingdon Police Department       13462               1979      Male   
10       Abingdon Police Department       13462               1980      Male   
11       Abingdon Police Department       13462               1983      Male   
12       Abingdon Police Department       13462               1960      Male   
13       Abingdon Police Department       13462               1986      Male   
14       Abingdon Police Department       13462               1977      Male   
15       Abingdon Police Department       13462               1967    Female   
16       Abingdon Police Department       13462               1984      Male   
17       Abingdon Police Department       13462               1976      Male   
18       Abingdon Police Department       13462               1978      Male   
19       Abingdon Police Department       13462               1963      Male   
20       Abingdon Police Department       13462               1983      Male   
21       Abingdon Police Department       13462               1978    Female   
22       Abingdon Police Department       13462               1987    Female   
23       Abingdon Police Department       13462               1967      Male   
24       Abingdon Police Department       13462               1982      Male   
25       Abingdon Police Department       13462               1977    Female   
26       Abingdon Police Department       13462               1966    Female   
27       Abingdon Police Department       13462               1982    Female   
28       Abingdon Police Department       13462               1979    Female   
29       Abingdon Police Department       13462               1963      Male   
...                             ...         ...                ...       ...   
2494633                Crete Police       13951               1987      Male   
2494634                Crete Police       13951               1986      Male   
2494635                Crete Police       13951               1980      Male   
2494636                Crete Police       13951               1979      Male   
2494637                Crete Police       13951               1980      Male   
2494638                Crete Police       13951               1973      Male   
2494639                Crete Police       13951               1970      Male   
2494640                Crete Police       13951               1951      Male   
2494641                Crete Police       13951               1985      Male   
2494642                Crete Police       13951               1986      Male   
2494643                Crete Police       13951               1952      Male   
2494644                Crete Police       13951               1965      Male   
2494645                Crete Police       13951               1977      Male   
2494646                Crete Police       13951               1963      Male   
2494647                Crete Police       13951               1979      Male   
2494648                Crete Police       13951               1979      Male   
2494649                Crete Police       13951               1970      Male   
2494650                Crete Police       13951               1981      Male   
2494651                Cr